In [1]:
# Import Plant

from model.plant_graph import GraphPlant
from model.plant_rearrangement import RearrangmentPlantV1, RearrangmentPlantV2
from model.tools import SystemSpecification
from model import StationNameType
from model import Vector

In [2]:
# Data generation

model_file_path = "../model.yaml"

model_file = open(model_file_path, "r")

spec = SystemSpecification(model_stream=model_file)

plant = GraphPlant(spec)

plant.import_config([
    (Vector(2,0), "InOut"),
    (Vector(2,1), "Robot1"),
    (Vector(2,2), "Press"),
    (Vector(1, 1), "PartsStorage"),
    (Vector(1,2), "Robot2"),

])

print("Original plant")
plant.print()

objective_plant = GraphPlant(spec)

objective_plant.import_config([
    (Vector(2,0), "InOut"),
    (Vector(2,1), "Robot1"),
    (Vector(1,1), "Press"),
    (Vector(2, 2), "PartsStorage"),
    (Vector(1,2), "Robot2"),

])

print("Objective plant")
objective_plant.print()

manipulated_plant = GraphPlant(spec)

sequence_list: list[tuple[StationNameType, Vector[int] | int, Vector[int] | int]] = []

Original plant
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|                 |        A        |        B        |        C        |        D        |        E        |
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|        0        |       None      |       None      |      InOut      |       None      |       None      |
|        1        |       None      |   PartsStorage  |      Robot1     |       None      |       None      |
|        2        |       None      |      Robot2     |      Press      |       None      |       None      |
|        3        |       None      |       None      |       None      |       None      |       None      |
|        4        |       None      |       None      |       None      |       None      |       None      |
+-----------------+-----------------+-----------------+-----------------+-----------------+--------------

In [3]:
# Print results function
def print_results():
    # Show the final plant and the objective plant
    print("Manipulated Plant final version")
    manipulated_plant.print()

    print("Sequence List")
    for item in sequence_list:
        print(item)

def map_location(value: Vector[int] | int) -> str:
    if isinstance(value, Vector):
        return f"{chr(ord('@')+value.x)}{value.y}"
    else:
        return f"SP{value + 1}"


def print_sequence():
    print("Sequence List")
    for index, item in enumerate(sequence_list):
        print(f"{index}: {item[0]} from {map_location(item[1])} to {map_location(item[2])}")

In [4]:
# Version 1

# Compare the two plants, create a 2D array of the differences

import itertools


manipulated_plant = RearrangmentPlantV1(spec)

manipulated_plant.import_config(plant.export_config())

equal = plant.grid_compare(objective_plant)


# In the first version of the algorithm we can only add or remove items from the plant from the right side.
# So, if a coordinate is different we need to remove all items at its right to reach that position.

# Storage buffer will be a linear array representing the storage places for stations transitions
# The first step would be to remove all items from the right until the first difference is reached in each row, to put them in the buffer, for each row
# Then the items in the buffer will be added to the plant in the right position


for x, y in itertools.product(
    range(spec.model.stations.grid.size.x), 
    range(spec.model.stations.grid.size.y)
    ):
    if not equal[y][x]:
        for i in range(spec.model.stations.grid.size.x - 1, x - 1, -1):
            if manipulated_plant.is_empty_coord(i, y):
                continue
            result = manipulated_plant.move_station_to_storage_buffer_coord(i, y)
            sequence_list.append((result[0], Vector(i, y), result[1]))


# Now we need to add the items in the buffer to the plant in the right position

# Compare the manipulated plant with the objective plant, to know the locations that require changes
# Iterate over the results, left to right, and add the items from the buffer to the manipulated plant

equal = manipulated_plant.grid_compare(objective_plant)

for x, y in itertools.product(
    range(spec.model.stations.grid.size.x), 
    range(spec.model.stations.grid.size.y)
    ):
    if not equal[y][x]:
        objective_station_name = objective_plant.get_station_name_coord(x, y)
        if objective_station_name is None:
            continue
        result = manipulated_plant.move_station_from_buffer_to_coord(
            objective_station_name, x, y
        )
        sequence_list.append((objective_station_name, result, Vector(x, y)))

print_sequence()

Sequence List
0: Robot1 from B1 to SP2
1: PartsStorage from A1 to SP3
2: Press from B2 to SP4
3: Press from SP3 to A1
4: Robot1 from SP1 to B1
5: PartsStorage from SP2 to B2


In [5]:
# Version 2

# The next version should come with limited storage capacity

# Compare the two plants, create a 2D array of the differences

StorageBufferNameType = str

manipulated_plant = RearrangmentPlantV2(spec, 3)

manipulated_plant.import_config(plant.export_config())

sequence_list: list[tuple[StationNameType, Vector[int] | int, Vector[int] | int]] = []

equal = plant.grid_compare(objective_plant)


# In the first version of the algorithm we can only add or remove items from the plant from the right side.
# So, if a coordinate is different we need to remove all items at its right to reach that position.

# Storage buffer will be a linear array representing the storage places for stations transitions
# The first step would be to remove all items from the right until the first difference is reached in each row, to put them in the buffer, for each row
# Then the items in the buffer will be added to the plant in the right position

for x, y in itertools.product(
    range(spec.model.stations.grid.size.x), 
    range(spec.model.stations.grid.size.y)
    ):
    if not equal[y][x]:
        for i in range(spec.model.stations.grid.size.x - 1, x - 1, -1):
            if manipulated_plant.is_empty_coord(i, y):
                continue
            result = manipulated_plant.move_station_to_storage_buffer_coord(i, y)
            sequence_list.append((result[0], Vector(i, y), result[1]))


# Now we need to add the items in the buffer to the plant in the right position

# Compare the manipulated plant with the objective plant, to know the locations that require changes
# Iterate over the results, left to right, and add the items from the buffer to the manipulated plant

equal = manipulated_plant.grid_compare(objective_plant)

for x, y in itertools.product(
    range(spec.model.stations.grid.size.x), 
    range(spec.model.stations.grid.size.y)
    ):
    if not equal[y][x]:
        objective_station_name = objective_plant.get_station_name_coord(x, y)
        result = manipulated_plant.move_station_from_buffer_to_coord(objective_station_name, x, y)
        sequence_list.append((objective_station_name, result, Vector(x, y)))

print_sequence()

Sequence List
0: Robot1 from B1 to SP2
1: PartsStorage from A1 to SP3
2: Press from B2 to SP4
3: Press from SP3 to A1
4: Robot1 from SP1 to B1
5: PartsStorage from SP2 to B2


The next version should be able to simplify the sequence list, by merging consecutive moves of the same station

It can be done something similar in three ways:

 - By post-processing the output of the V2 version, merging consecutive moves of the same station. This is the simplest way, but it is not the most efficient one.
 - By, at any time, when a station is moved to the storage buffer, check if can be fitted directly in other row, by brute force. This way is also simple, but will slow down the algorithm.
 - Another option is, when the last station from a row is moved to the storage buffer and the remaining stations are in right place, save the station name and position in a list named ready_to_install. Later, whenever a station is moved to the storage buffer, check if it is in the ready_to_install list, then move it to the right place. 

In [6]:
# Version 3

# Compare the two plants, create a 2D array of the differences

StorageBufferNameType = str

manipulated_plant = RearrangmentPlantV2(spec, 3)

manipulated_plant.import_config(plant.export_config())

sequence_list: list[tuple[StationNameType, Vector[int] | int, Vector[int] | int]] = []

equal = plant.grid_compare(objective_plant)

ready_positions: dict[StationNameType, Vector[int]] = {}

# In the first version of the algorithm we can only add or remove items from the plant from the right side.
# So, if a coordinate is different we need to remove all items at its right to reach that position.

# Storage buffer will be a linear array representing the storage places for stations transitions
# The first step would be to remove all items from the right until the first difference is reached in each row, to put them in the buffer, for each row
# Then the items in the buffer will be added to the plant in the right position

for y in range(spec.model.stations.grid.size.y):
    for x in range(spec.model.stations.grid.size.x):
        if not equal[y][x]:

            for i in range(spec.model.stations.grid.size.x - 1,  x - 1, -1):

                if manipulated_plant.is_empty_coord(i, y):
                    continue

                focus_station_name = manipulated_plant.get_station_name_coord(i, y)

                if focus_station_name in ready_positions:
                    ready_info = ready_positions.pop(focus_station_name)

                    result = manipulated_plant.move_coord_to_another_coord(i, y, ready_info.x, ready_info.y)
                    sequence_list.append((result, Vector(i, y), Vector(ready_info.x, ready_info.y)))

                    # Update the ready positions with the station at the right of the just moved station if any
                    if ready_info.x + 1 < spec.model.stations.grid.size.x:
                        
                        objective_station = objective_plant.get_station_or_null_coord(ready_info.x + 1, ready_info.y)

                        if objective_station is not None:
                            ready_positions[objective_station.name] = Vector(ready_info.x + 1, ready_info.y)

                else:
                    result = manipulated_plant.move_station_to_storage_buffer_coord(i, y)
                    sequence_list.append((result[0], Vector(i, y), result[1]))
            
            ready_positions[objective_plant.get_station_name_coord(i, y)] = Vector(i, y)
            break


# Now we need to add the items in the buffer to the plant in the right position

# Compare the manipulated plant with the objective plant, to know the locations that require changes
# Iterate over the results, left to right, and add the items from the buffer to the manipulated plant

equal = manipulated_plant.grid_compare(objective_plant)

for x, y in itertools.product(
    range(spec.model.stations.grid.size.x), 
    range(spec.model.stations.grid.size.y)
    ):
    if not equal[y][x]:
        objective_station_name = objective_plant.get_station_name_coord(x, y)
        result = manipulated_plant.move_station_from_buffer_to_coord(objective_station_name, x, y)
        sequence_list.append((objective_station_name, result, Vector(x, y)))


print_sequence()

Sequence List
0: Robot1 from B1 to SP2
1: PartsStorage from A1 to SP3
2: Press from B2 to A1
3: Robot1 from SP1 to B1
4: PartsStorage from SP2 to B2


In [7]:
# Version 4
"""
If the grid stations can be moved from both sides of the plant, a brute force graph based aproach can be used to find the optimal solution

The graph would be a directed graph, where each node would represent a plant status and each edge would represent a station movement.

First, the graph would start with initial plant status as the root node, and the objetive plant status as the target node. We would then use a BFS algorithm to find the shortest path between the two nodes. Using bfs, once the objetive plant status is reached, we would have the shortest path.

The stations can only be moved to an storage place or to the right final position

"""

from model.plant_rearrangement import RearrangmentPlantV3


StorageBufferNameType = str

manipulated_plant = RearrangmentPlantV3(spec, 3)

manipulated_plant.import_config(plant.export_config())

sequence_list: list[tuple[StationNameType, Vector | StorageBufferNameType, Vector | StorageBufferNameType]] = []

# We have to compare the manipulated plant with the objective plant, to know the locations that require changes. Also, if a station not in the right position is surrounded by stations at both sides, those stations would be allowed to change their location.

In [8]:
# Directed graph class

from typing import Literal

from traitlets import Int
from graph import DirectedGraphEdge, DirectedGraphNode
from model.plant_rearrangement import RearrangmentPlantV3


class RearrangementNode(DirectedGraphNode):
    def __init__(self, plant: RearrangmentPlantV3):
        super().__init__()
        self.plant = plant

class RearrangementEdge(DirectedGraphEdge):
    def __init__(self, station_name, to: Vector[int] | int):
        """Station movement from actual position to "to" place

        "to" can be a vector representing a new position in the 2D grid plant or an integer representing a storage buffer position

        Args:
            station_name (_type_): _description_
            to (Vector[int] | int): _description_
        """
        super().__init__()
        self.station = station_name
        self.to = to


In [9]:
import itertools
from typing import Any


def check_isolated_falsy_on_grid(grid: list[list[Any]]) -> None:
    for x, y in itertools.product(range(len(grid)), range(len(grid[0]))):
        if x > 0 and x < len(grid) - 1:
            if grid[x - 1][y] is False and grid[x + 1][y] is False:
                grid[x][y] = False


In [10]:
# Start the graph with the first node

import copy
from operator import le


root_node = RearrangementNode(copy.deepcopy(manipulated_plant))

# From root node we are going to add all the possible movements as edges

def process_node(node: RearrangementNode):
    """This function deep another level in the BFS algorithm

    Args:
        node (RearrangementNode): _description_
    """

    # First, we need to compare the node plant with the original plant, to know the station that can be moved

    equal = node.plant.grid_compare(plant)

    check_isolated_falsy_on_grid(equal)

    move_required_stations: set[StationNameType] = set()

    for x, y in itertools.product(
        range(spec.model.stations.grid.size.x), 
        range(spec.model.stations.grid.size.y)
        ):
        
        if not equal[y][x]:
            station_name = node.plant.get_station_name_or_null_coord(x, y)
            if station_name is not None:
                move_required_stations.add(station_name)

    # Now we are going to iterate over the stations that can be moved and create movement edges for each one

    for station_name in move_required_stations:
        objective_position = objective_plant.station_position(station_name) 
        
        
        
    # Iterate over the results and create a new edge and node for each station that can be moved, with a copy of the plant

    
    